In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import torch

In [23]:
from lib.actor.LeeNet import LeeNetActor
from lib.models.backbone.plscore_RMT import PLScoreRMT
from lib.models.LeeNet.score_pureRMT_mlp import ScorePureRMTMLP
from lib.models.head.mlp import MLP
from lib.models.layer.patch_embed import PatchEmbed
from lib.models.layer.score import PLScoreLayerUseConv
from lib.trainer.LeeNet_trainer import LeeNetTrainer
from lib.utils.base_funtion import build_dataloaders, get_optimizer_scheduler
from lib.config.cfg_loader import env_setting
from torch.nn.functional import l1_loss
from torch.nn import BCEWithLogitsLoss
from lib.utils.box_ops import giou_loss
from lib.utils.focal_loss import FocalLoss
import torch


def build_model(cfg):
    backbone = PLScoreRMT(cfg=cfg)
    head = MLP(input_dim=10 * cfg.model.pureRMT.embed_dim[-1], hidden_dim=cfg.model.pureRMT.embed_dim[-1], output_dim=4, num_layers=2)
    model = ScorePureRMTMLP(backbone, head, cfg)
    return model

In [31]:
checkpoint_path = "/media/star/data/Leezed/workspace/LeeNet/checkpoints/LeeNet_score_RMT_MLP00001/ScorePureRMTMLP_ep0636.pth.tar"
checkpoint_dict = torch.load(checkpoint_path)['net']


cfg = env_setting(cfg_name='plscore_pureRMT_mlp00001.yaml')

loader_train, loader_val = build_dataloaders(cfg)

net = build_model(cfg)



model_dict =  net.state_dict()
state_dict = {k:v for k,v in checkpoint_dict.items() if k in model_dict.keys()}
print(state_dict.keys())
model_dict.update(state_dict)
net.load_state_dict(model_dict)

dict_keys(['backbone.patch_embed.proj.weight', 'backbone.patch_embed.proj.bias', 'backbone.score.conv1.conv.weight', 'backbone.score.conv1.bn.weight', 'backbone.score.conv1.bn.bias', 'backbone.score.conv1.bn.running_mean', 'backbone.score.conv1.bn.running_var', 'backbone.score.conv1.bn.num_batches_tracked', 'backbone.score.conv2.conv.weight', 'backbone.score.conv2.bn.weight', 'backbone.score.conv2.bn.bias', 'backbone.score.conv2.bn.running_mean', 'backbone.score.conv2.bn.running_var', 'backbone.score.conv2.bn.num_batches_tracked', 'backbone.RMT_layers.0.Relpos.angle', 'backbone.RMT_layers.0.Relpos.decay', 'backbone.RMT_layers.0.blocks.0.retention_layer_norm.weight', 'backbone.RMT_layers.0.blocks.0.retention_layer_norm.bias', 'backbone.RMT_layers.0.blocks.0.retention.q_proj.weight', 'backbone.RMT_layers.0.blocks.0.retention.q_proj.bias', 'backbone.RMT_layers.0.blocks.0.retention.k_proj.weight', 'backbone.RMT_layers.0.blocks.0.retention.k_proj.bias', 'backbone.RMT_layers.0.blocks.0.reten

<All keys matched successfully>

In [32]:


focal_loss = FocalLoss()
objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

optimizer, lr_scheduler = get_optimizer_scheduler(net, cfg)

trainer = LeeNetTrainer(actor=actor, loaders=[loader_train, loader_val], optimizer=optimizer, lr_scheduler=lr_scheduler, cfg=cfg)

trainer.train(cfg.train.epoch)

Epoch Time: 0:00:28.737890
Avg Data Time: 23.61452
Avg GPU Trans Time: 0.15747
Avg Forward Time: 4.96590
Epoch Time: 0:00:29.986429
Avg Data Time: 11.83567
Avg GPU Trans Time: 0.13366
Avg Forward Time: 3.02388
Epoch Time: 0:00:30.686485
Avg Data Time: 7.91037
Avg GPU Trans Time: 0.12294
Avg Forward Time: 2.19551
Epoch Time: 0:00:31.397435
Avg Data Time: 5.94835
Avg GPU Trans Time: 0.11910
Avg Forward Time: 1.78191
Epoch Time: 0:00:32.102973
Avg Data Time: 4.77290
Avg GPU Trans Time: 0.11453
Avg Forward Time: 1.53317
Epoch Time: 0:00:32.827237
Avg Data Time: 3.98658
Avg GPU Trans Time: 0.11214
Avg Forward Time: 1.37249
Epoch Time: 0:00:33.663095
Avg Data Time: 3.42681
Avg GPU Trans Time: 0.12669
Avg Forward Time: 1.25552
Epoch Time: 0:00:34.363508
Avg Data Time: 3.00572
Avg GPU Trans Time: 0.12351
Avg Forward Time: 1.16621
Epoch Time: 0:00:48.751745
Avg Data Time: 4.19230
Avg GPU Trans Time: 0.12143
Avg Forward Time: 1.10313
[train: 1, 10 / 468] FPS: 25.8 (162.7)  ,  DataTime: 3.779 (0.

KeyboardInterrupt: 